In [2]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.6 MB/s eta 0:00:00


In [78]:
import pandas as pd
import contractions
import re
from string import punctuation
from collections import Counter

import nltk
from nltk.corpus import stopwords

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

In [38]:
df = pd.read_csv('/content/drive/MyDrive/cyberbullying_tweets.csv')
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


# EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [ ]:
df['cyberbullying_type'].value_counts()

,count
cyberbullying_type,
religion,7998
age,7992
gender,7973
ethnicity,7961
not_cyberbullying,7945
other_cyberbullying,7823


# Data Cleaning

In [39]:
df['tweet_text'] = df['tweet_text'].apply(lambda x: x.lower()) #**

df['tweet_text'] = df['tweet_text'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()])) #**

df['tweet_text'] = df['tweet_text'].apply(lambda x: re.sub('\[.*?\]', '', x))

df['tweet_text'] = df['tweet_text'].apply(lambda x: re.sub('https?://\S+|www\.\S+', '', x)) #**

df['tweet_text'] = df['tweet_text'].apply(lambda x: re.sub('<.*?>+', '', x))

df['tweet_text'] = df['tweet_text'].apply(lambda x: re.sub('[%s]' % re.escape(punctuation), '', x)) #**

df['tweet_text'] = df['tweet_text'].apply(lambda x: re.sub('\n', '', x))

df['tweet_text'] = df['tweet_text'].apply(lambda x: re.sub('\w*\d\w*', '', x))

In [70]:
df['tweet_text']

,tweet_text
0,word katandandre food crapilicious mkr
1,aussietv white mkr theblock imacelebrityau tod...
2,xochitlsuckkks classy whore red velvet cupcake
3,jasongio meh thanks head concerned another ang...
4,rudhoeenglish isi account pretending kurdish a...
...,...
47687,black people expected anything depended anythi...
47688,turner withhold disappointment turner called c...
47689,swear god dumb nigger bitch got bleach hair re...
47690,yea fuck rt therealexel nigger fucking unfollo...


In [ ]:
df['tweet_text']

,tweet_text
0,"In other words #katandandre, your food was cra..."
1,Why is #aussietv so white? #MKR #theblock #ImA...
2,@XochitlSuckkks a classy whore? Or more red ve...
3,"@Jason_Gio meh. :P thanks for the heads up, b..."
4,@RudhoeEnglish This is an ISIS account pretend...
...,...
47687,"Black ppl aren't expected to do anything, depe..."
47688,Turner did not withhold his disappointment. Tu...
47689,I swear to God. This dumb nigger bitch. I have...
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...


# Tokenization

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

Vectorizer = CountVectorizer()

tok = Vectorizer.build_tokenizer()

df['tweet_text'] = df['tweet_text'].apply(tok)

# Counter

In [47]:
top = Counter([word for sentences in df['tweet_text'] for word in sentences])

top_10 = top.most_common(10)
top_10

[('the', 30953),
 ('you', 26897),
 ('to', 24697),
 ('is', 22293),
 ('and', 21576),
 ('not', 17504),
 ('of', 15504),
 ('in', 14929),
 ('are', 13986),
 ('that', 13218)]

# remove stopword

In [48]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
stops = set(stopwords.words('english'))

def remove_stops(list):
  return [w for w in list if w not in stops]

df['tweet_text'] = df['tweet_text'].apply(lambda x: remove_stops(x))

In [54]:
top = Counter([word for sentences in df['tweet_text'] for word in sentences])

top_10 = top.most_common(10)
top_10

[('school', 8738),
 ('like', 5865),
 ('fuck', 5808),
 ('people', 5412),
 ('dumb', 5341),
 ('high', 5098),
 ('bullied', 4666),
 ('nigger', 4364),
 ('rt', 4265),
 ('rape', 4238)]

# Most commen words in each class

In [55]:
df.head()

,tweet_text,cyberbullying_type
0,"[words, katandandre, food, crapilicious, mkr]",not_cyberbullying
1,"[aussietv, white, mkr, theblock, imacelebritya...",not_cyberbullying
2,"[xochitlsuckkks, classy, whore, red, velvet, c...",not_cyberbullying
3,"[jasongio, meh, thanks, heads, concerned, anot...",not_cyberbullying
4,"[rudhoeenglish, isis, account, pretending, kur...",not_cyberbullying


In [56]:
df['cyberbullying_type'].value_counts()

,count
cyberbullying_type,
religion,7998
age,7992
gender,7973
ethnicity,7961
not_cyberbullying,7945
other_cyberbullying,7823


In [58]:
filtered_religion = df[df['cyberbullying_type'] == 'religion']

top = Counter([word for sentences in filtered_religion['tweet_text'] for word in sentences])

print('Most common words in religion type cyberbullying:')
top_10 = top.most_common(10)
top_10

Most common words in religion type cyberbullying


[('muslims', 2371),
 ('muslim', 2270),
 ('idiot', 1731),
 ('christian', 1481),
 ('idiots', 1333),
 ('islamic', 1253),
 ('like', 1234),
 ('islam', 1179),
 ('people', 1178),
 ('terrorism', 1101)]

In [ ]:
filtered_religion = df[df['cyberbullying_type'] == 'religion']

top = Counter([word for sentences in filtered_religion['tweet_text'] for word in sentences])

print('Most common words in religion type cyberbullying:')
top_10 = top.most_common(10)
top_10

In [59]:
filtered_age = df[df['cyberbullying_type'] == 'age']

top = Counter([word for sentences in filtered_age['tweet_text'] for word in sentences])

print('Most common words in age type cyberbullying:')
top_10 = top.most_common(10)
top_10

Most common words in age type cyberbullying:


[('school', 8186),
 ('high', 4940),
 ('bullied', 4370),
 ('bully', 2703),
 ('girls', 2342),
 ('girl', 2322),
 ('like', 1847),
 ('bullies', 1623),
 ('people', 1002),
 ('would', 953)]

In [60]:
filtered_gender = df[df['cyberbullying_type'] == 'gender']

top = Counter([word for sentences in filtered_gender['tweet_text'] for word in sentences])

print('Most common words in gender type cyberbullying:')
top_10 = top.most_common(10)
top_10

Most common words in gender type cyberbullying:


[('rape', 4085),
 ('gay', 3700),
 ('jokes', 3407),
 ('joke', 1887),
 ('rt', 1245),
 ('people', 1090),
 ('call', 987),
 ('sexist', 964),
 ('like', 921),
 ('bitch', 853)]

In [62]:
filtered_other_cyberbullying = df[df['cyberbullying_type'] == 'other_cyberbullying']

top = Counter([word for sentences in filtered_other_cyberbullying['tweet_text'] for word in sentences])

print('Most common words in other cyberbullying:')
top_10 = top.most_common(10)
top_10

Most common words in other cyberbullying:


[('rt', 886),
 ('bully', 527),
 ('like', 470),
 ('people', 456),
 ('fucking', 370),
 ('get', 352),
 ('would', 310),
 ('know', 288),
 ('mkr', 282),
 ('think', 248)]

# Lemmization

In [64]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [67]:
from nltk.stem import WordNetLemmatizer as wnl

def limmization(list):
  return [wnl().lemmatize(word) for word in list]

df['tweet_text'] = df['tweet_text'].apply(lambda x: limmization(x))

# Joining the words after cleaning

In [69]:
df['tweet_text'] = df['tweet_text'].apply(lambda x: ' '.join(x))

# Count Vectorizer

In [73]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(df['tweet_text'])
y = df['cyberbullying_type']

In [74]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [75]:
vectorizer.get_feature_names_out()

array(['aa', 'aaa', 'aaaa', ..., '𝕨𝕠𝕦𝕝𝕕', '𝕫𝕠𝕖𝕪', '𝚎𝚕𝚞𝚜𝚒𝚟𝚎'], dtype=object)

# Train, test splitting

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

# Modeling

## SVM

In [77]:
from sklearn import svm


clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [82]:
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Confusion Matrix: ", confusion)

Accuracy:  0.8337351923681727
Confusion Matrix:  [[1556    1    4   20   17    5]
 [   4 1561    5    7   21    5]
 [   1    3 1266   93  159    9]
 [  18    6   48  766  733   53]
 [   3    6   66  198 1336    3]
 [   3    4    4   52   35 1468]]


## Decision Tree

In [83]:
from sklearn import tree

clf_tree = tree.DecisionTreeClassifier()
clf_tree = clf_tree.fit(X_train, y_train)

In [84]:
y_pred = clf_tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Confusion Matrix: ", confusion)

Accuracy:  0.7831009539784044
Confusion Matrix:  [[1544    2    4   31   20    2]
 [   6 1547    9   20   14    7]
 [   4    4 1283  124  112    4]
 [  34    6  102  803  613   66]
 [  14    7  141  590  852    8]
 [   1    8   14   60   42 1441]]
